In [ ]:
import importlib
import chembl_connect as cc
importlib.reload(cc)
import pandas as pd
import json 

df = pd.read_csv("drug.csv")
drug_names = df["Drug"].dropna().unique().tolist()

test = cc.main(drug_list= drug_names, protein_name="Dopamine transporter")

Searching for ChEMBL ID for target: 'Dopamine transporter'...
{'q': 'Dopamine transporter', 'limit': 1, 'format': 'json', 'organism': 'Homo sapiens'}
b'{"page_meta": {"limit": 1, "next": "/chembl/api/data/target/search.json?limit=1&offset=1&q=Dopamine+transporter&organism=Homo+sapiens", "offset": 0, "previous": null, "total_count": 236}, "targets": [{"cross_references": [], "organism": "Homo sapiens", "pref_name": "Dopamine transporter", "score": 19.0, "species_group_flag": false, "target_chembl_id": "CHEMBL238", "target_components": [{"accession": "Q01959", "component_description": "Sodium-dependent dopamine transporter", "component_id": 346, "component_type": "PROTEIN", "relationship": "SINGLE PROTEIN", "target_component_synonyms": [{"component_synonym": "DAT", "syn_type": "UNIPROT"}, {"component_synonym": "DAT1", "syn_type": "GENE_SYMBOL_OTHER"}, {"component_synonym": "DA transporter", "syn_type": "UNIPROT"}, {"component_synonym": "SLC6A3", "syn_type": "GENE_SYMBOL"}, {"component_sy